# Improving Spell Checker
by Vance Piscitelli for CS 482  
  
My program starts with a spelling corrector written by Peter Norvig which can correct a word to what the user might have meant to type. I improve it by keeping track of the past n words that immediately follow in the provided text file. 
## Brief Summary of How it Works  
<b>word Z:</b> A misspelled word that needs to be corrected.  
<b>word X:</b> A correctly spelt word that comes immediately before word Z.  
<b>word Y:</b> A correctly spelt word that has frequently come after word X.  
When correcting word Z, if it is similar enough to word Y, then word Y will be used as the correction to Word Z.


<b>Original Spelling Corrector</b>

In [1]:
# Spelling Corrector
# this code is by Peter Norvig 
# norvig.com/spell-correct.html

# this code can take a word and correct it. Sometimes the word is already correct and it ought to
# stay the same, or if it is wrong, it will change it to what has the highest probability of being
# correct. One issue with his solution is that he doesn't account for words that typically come
# after other words.

import re, collections

def words(text): return re.findall('[a-z]+', text.lower()) 

def train(features):
    model = collections.defaultdict(lambda: 1)
    for f in features:
        model[f] += 1
    return model

# trains the model based of off the specified text file
NWORDS = train(words(file('big.txt').read()))

# a-z plus some special characters that I added
alphabet = 'abcdeéfghijklmnopqrstuvwxyz'

# returns words that are 1 edit away
def edits1(word):
   splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]
   deletes    = [a + b[1:] for a, b in splits if b]
   transposes = [a + b[1] + b[0] + b[2:] for a, b in splits if len(b)>1]
   replaces   = [a + c + b[1:] for a, b in splits for c in alphabet if b]
   inserts    = [a + c + b     for a, b in splits for c in alphabet]
   return set(deletes + transposes + replaces + inserts)

# returns words that are 2 edits away
def known_edits2(word):
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1) if e2 in NWORDS)

# returns the word if it exists in the list of known words
def known(words): return set(w for w in words if w in NWORDS)

def correct(word):
    candidates = known([word]) or known(edits1(word)) or known_edits2(word) or [word]
    return max(candidates, key=NWORDS.get)

In [2]:
correct("appees")

'apples'

<b>Sentence Correction:</b>  
Instead of working on just a single word, I add the ability to correct a whole sentence. The issue with this is that each word is still being corrected on an individual level with no respect for the other words. Maybe the user typed 'qat' but meant the word 'bat' which based on the context of the surrounding words would be obvious. However, the spell checker might end up selecting 'cat' as being the correction.

In [3]:
def sentenceCorrect(string):
    word = [] # holds a word
    sentence = [] # holds the entire sentence in correct format
    
    # convert input into a list of words
    for i in range(len(string)):
        if string[i] == ' ': # store word once a space is found
            w = ''.join(word)
            sentence.append(w)
            word = []
        else: # if not a space yet, keep adding on characters
            word.append(string[i])
    
    # get last word added into sentence
    if word != []:
        w = ''.join(word)
        sentence.append(w)
        word = []
    
    # correct each word in sentence
    for i in range(len(sentence)):
        print correct(sentence[i]), # corrects sentence and then prints

In [4]:
sentenceCorrect("fsh fod thouht")

fish for thought


<b>Creating Word Objects</b>  
In order to make the spell checker actually make corrections that takes the previous word into consideration, I've got to keep track of what words tend to come after other words. For example, if in the text file the word 'tree' comes after the word 'apple' eight times and the word 'sauce' comes after the word 'apple' only two times, the program should keep track of that. In order to keep track of this information, I created a word object that has a list of the past n words that occurred immediately after the given word.

In [5]:
# this will be a word that will be stored in a dictionary that keeps track of the past n words used
# right after it. By only keeping a limited number of words, the spell checker can change as the user 
# changes how he/she writes over time.
class Word(object):
    name = []
    wordsAfter = []
    
    def __init__(self, word):
        self.name = word
        
    def addWordsAfter(self, word):
        self.wordsAfter.append(word)
        
        # for now, a limit of the past n words used right after.
        if len(self.wordsAfter) > 100:
            self.wordsAfter.pop(0)
            
    def printWordsAfter(self):
        for i in range(len(self.wordsAfter)):
            print self.wordsAfter[i]

<b>Adding a Dictionary</b>  
An individual word is not very helpful on its own and the text file has thousands of different words so I used a dictionary to hold all of the word objects. This results in one word object for each different word in the text file along with the words that came immediately after it each time a word was encountered.

In [6]:
# keeps track of each word and the words most commonly used after it
class myDictionary(object):
    filename = ""
    fDict = dict()
    
    def __init__(self, fn):
        filename = fn
        
        f = open(filename, 'r')
        words = map(lambda l: l.split(), f.readlines()) # reads in file and stores each word in words
        
        print len(words)
        print len(words[0])
        # convert all words to lower case
        for i in range(len(words)):
            for j in range(len(words[i])):
                words[i][j] = words[i][j].lower()
                if words[i][j].count('\\n') > 0:
                    words[i][j].pop()
                    words[i][j].pop()
        
        
        
        iMax = len(words)
        for i in range(iMax):
            jMax = len(words[i])
            for j in range(jMax):
                    
                if words[i][j] not in self.fDict: # if word from file is not in dictionary yet   
                    self.fDict[words[i][j]] = Word(words[i][j]) # add it to the dictionary using Word class as def above
                    self.fDict[words[i][j]].wordsAfter = []
                    
                # add the word that comes after current word to list
                if j < jMax - 1: # if not at last word on line
                    self.fDict[words[i][j]].addWordsAfter(words[i][j + 1])

<b>Testing the Dictionary</b>  
At this point, my program can read through the text file and it keeps track of the past n words that occured after each word. Certain words tend to occur more frequently than other words which the program will use later on. The following code shows that it is indeed keeping track of that information.

In [7]:
Dictionary = myDictionary("bigEdit.txt")

print "Word:",
print Dictionary.fDict["and"].name
print "Words that came immediately after:"
print Dictionary.fDict["and"].wordsAfter
print
print "Word:",
print Dictionary.fDict["or"].name
print "Words that came immediately after:"
print Dictionary.fDict["or"].wordsAfter

128358
10
Word: and
Words that came immediately after:
['independent', 'entirely', 'therefore', 'the', 'so', 'so', 'so', 'an', 'not', 'free', 'cause,', 'the', 'cannot', 'is', 'effect', 'can', 'all', 'inevitability.', 'content,', 'we', 'so', 'we', 'as', 'electricity,', 'so', 'in', 'above', 'in', 'so', 'newton', 'no', 'define', 'then', 'the', 'dissecting', 'enters', 'if', 'of', 'not', 'proved,', 'proved', 'that', 'that', 'such', 'yet', 'geology,', 'the', 'accused', 'stubborn', 'the', 'theology', 'accuses', 'stifles', 'regret', 'in', 'newton,', 'he', 'all', 'church', 'church', 'of', 'cause', 'on', 'to', 'to', 'peace', 'editing', 'how', 'just', 'analyzed,', 'organizations', 'wyoming.', 'fund', "don't", 'fund-raising', 'even', 'accept', 'distribute', 'proofread', 'any', 'the', 'expenses,', '[2]', 'exclusions', 'you', 'hold', 'its', 'agents,', 'any', 'distribution', 'expense,', 'all', 'underline', 'additional', 'replacement', 'to', 'licensed', 'trailer', 'may', 'to']

Word: or
Words that cam

<b>Generating Random Sentences</b>  
Since I can keep track of the words that come after, I can randomly generate sentences that are somewhat understandable. All the program has to do is pick a random word from the list of words that have come after each word in the sentence. It runs into some issues where it encounters words that were infrequently used, such as only once and that can cause it to end up on a path with only one option each time. This isn't helpful with the main purpose of the program but it was still slightly interesting so I tried it out anyway.

In [8]:
from random import randrange

currentWord = "the" # a word to start the sentence with

print "Randomly generated sentence:"

# prints 15 words
for i in range(15):
    if currentWord != '\n': # ignore words that are just a new line character
        print Dictionary.fDict[currentWord].name,
    
    numWordsAfter = len(Dictionary.fDict[currentWord].wordsAfter)
    if numWordsAfter > 0: # randomly pick the next word
        currentWord = Dictionary.fDict[currentWord].wordsAfter[randrange(len(Dictionary.fDict[currentWord].wordsAfter))]
    else: # to stop it from having an error, just give it a new word to continue with
        currentWord = "so"

Randomly generated sentence:
the goal is one man who have had a moment given to project gutenberg-tm ebooks


<b>Improving the Spelling Corrector</b>  
Now I actually change the spelling corrector to prefer a candidate for correction if it has occurred before. This is done by counting how many times a candidate appears in the list of words that have occurred after the previous word. Most of the code is still the same as before but my changes do make a difference as demonstrated below.

In [9]:
# Improved Spelling Corrector
# edited version of code originally by Peter Norvig 
# norvig.com/spell-correct.html

def wordsB(text): return re.findall('[a-z]+', text.lower()) 

def trainB(features):
    model = collections.defaultdict(lambda: 1)
    for f in features:
        model[f] += 1
    return model

NWORDSB = train(wordsB(file('bigEdit.txt').read()))

alphabet = 'abcdeéfghijklmnopqrstuvwxyz'

def edits1B(word):
   splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]
   deletes    = [a + b[1:] for a, b in splits if b]
   transposes = [a + b[1] + b[0] + b[2:] for a, b in splits if len(b)>1]
   replaces   = [a + c + b[1:] for a, b in splits for c in alphabet if b]
   inserts    = [a + c + b     for a, b in splits for c in alphabet]
   return set(deletes + transposes + replaces + inserts)

def known_edits2B(word):
    return set(e2 for e1 in edits1B(word) for e2 in edits1B(e1) if e2 in NWORDSB)

def knownB(words): return set(w for w in words if w in NWORDSB)

def correctB(word):
    candidates = knownB([word]) or knownB(edits1B(word)) or known_edits2B(word) or [word]
    return max(candidates, key=NWORDSB.get)

def correctImprovedB(word, previous):
    # get the potential candidates
    candidates = knownB([word]) or knownB(edits1B(word)) or known_edits2B(word) or [word]
    candidateList = list(candidates)
    # check if any of them have been used before after previous word
    cCount = []
    for i in range(len(candidateList)):
        wA = Dictionary.fDict[previous].wordsAfter
        c = candidateList[i]
        amount = wA.count(c)
        cCount.append(amount)
    
    # get the position of the candidate that has the highest occurance
    maxCount = max(cCount)
    maxPos = 0
    for i in range(len(cCount)):
        if cCount[i] == maxCount:
            maxPos = i
    
    # if the highest occurance is >1, then return it
    if Dictionary.fDict[previous].wordsAfter.count(candidateList[maxPos]) > 0:
        return candidateList[maxPos]
    else: # otherwise, just return the closest word
        return max(candidates, key=NWORDSB.get)

def sentenceCorrectB(string, append):
    word = [] # holds a word
    sentence = [] # holds the entire sentence in correct format
    
    # convert input into a list of words
    for i in range(len(string)):
        if string[i] == ' ': # store word once a space is found
            w = ''.join(word)
            sentence.append(w)
            word = []
        else: # if not a space yet, keep adding on characters
            word.append(string[i])
    
    # get last word added into sentence
    if word != []:
        w = ''.join(word)
        sentence.append(w)
        word = []
    
    # correct each word in sentence
    # note that the corrected word is added back into the sentence so the next word can use it
    for i in range(len(sentence)):
        if i == 0: # correct the first word
            sentence[i] = correctB(sentence[i]) 
        else: # correct the remaining words based on the previous words
            sentence[i] = correctImprovedB(sentence[i], sentence [i - 1])
            
        print sentence[i],
        
    if append == True:
        with open("bigEdit.txt", 'a') as f:
            f.write(sentence)

<b>Simple Test</b>  
Now, to run a simple test on it with a variation of a sentence that occurred in the text file:  
<b>Original sentence:</b> "A Frenchman or Russian could not have written that"  
<b>Misspelled:</b> "A Frenchnan zor Russean coul iot habe writen ohat"  
While the original spelling corrector managed to fix most of the issues, my improved version did a little better, at least in this example.  
Note: boolean value indicates whether the result should be appended to the file. Used later.

In [10]:
sentence = "A Frenchnan zor Russean coul iot habe writen ohat"
sentenceCorrect(sentence)
print
sentenceCorrectB(sentence, False)

a frenchman for russian could it have written that
a frenchman or russian could not have written that


<b>Appending</b>  
Now that the sentence is corrected, it could be appended to the text file and each time to program is run, it will start preferring words that are more commonly used by the user. For example, if this was a text messenger, and the user frequently sent "Hey, how are you?" then that would become a more likely correction if the user accidentally typed something such as "Heu, hoq are you?" as opposed to an incorrect correction like "Hen, hot are you?". All that needs to be done is change the boolean value in the sentenceCorrectB() function to True and it will start appending to the file and then just update Dictionary and it will have the most recent changes.

<b>Future Work</b>  
If I was to continue working on this, I would first start by doing more testing to see how much of an improvement it is actually doing as compared to the previous model. I would also change how it keeps track of the words that are commonly used after each word. Right now it is simply a queue of the past n words that were used which works but takes up a lot of space. Lastly, how it decides which word to pick could also be improved but as I already mentioned, I would need better testing in order to figure out how to best improve it.